In [27]:
from abc import ABC, abstractmethod
import gymnasium as gym
import random
import numpy as np
import torch
import torch.nn as nn
from collections import OrderedDict
import math

In [28]:
### MLP
class Multi_Layer_Perceptron(nn.Sequential):
    def __init__(self, input_dim, intern_dim, output_dim, depth = 2, isBiased = False):
        
        dict = OrderedDict([("input",nn.Linear(input_dim,intern_dim, bias=isBiased))])
        for i in range(depth):
            dict.update({str(i) : nn.Linear(intern_dim,intern_dim,bias=isBiased)})
        dict.update({"output" : nn.Linear(intern_dim,output_dim,bias=isBiased)})

        super().__init__(dict)

        self.reset_init_weights_biases(0) # so that we do not use a default initialization

    def reset_init_weights_biases(self, norm = None):
        for layer in self.children():
            if norm == None:
                stdv = 1. / math.sqrt(layer.weight.size(1))
            else :
                stdv = norm
            
            layer.weight.data.uniform_(-stdv, stdv)
            if layer.bias is not None:
                layer.bias.data.uniform_(-stdv, stdv)

In [29]:
### CustomLoss DQN
class DQN_Loss(nn.Module):
    def __init__(self):
        super(DQN_Loss, self).__init__()

    def forward(self, input, target):
        loss = (target - input)**2
        return loss

In [30]:
class Agent(ABC):

    @abstractmethod
    def observe(self, state, action, next_state, reward):
        pass

    @abstractmethod
    def select_action(self, state):
        pass
    
    @abstractmethod
    def update(self):
        pass

    @abstractmethod
    def train(self, episodes, debug_mode=False):
        pass

    def __init__(self, id, env):
        self.id = id
        self.env = env
                

In [31]:
class RandomAgent(Agent):
    def observe(self, state, action, next_state, reward):
        pass
        
    def select_action(self, state):
        return random.randint(0,1)
        
        
    def update(self):
        pass

    def train(self, episodes, debug_mode=False):
        episodesHistory = np.zeros((episodes))
        
        for i in range(episodes):
            if(debug_mode) : print("Episode: "+str(i+1)+" starts")
            newSeed = random.randint(0,100000)
            state,_ = self.env.reset(seed = newSeed)
            done = False
            episode_reward = 0
            
            while not done:
                                        
                action = self.select_action(state)
                next_state, reward, terminated, truncated, _ = self.env.step(action)
                self.observe(state,action,next_state,reward)
                self.update()

                episode_reward += reward
                state = next_state
                done = terminated or truncated

            episodesHistory[i] = episode_reward
            if(debug_mode) : print("Episode "+str(i+1)+ " , Reward: "+str(episode_reward))
        print(episodesHistory[:])

In [88]:
class DQNAgent(Agent):

    def __init__(self, id, epsilon, Q = Multi_Layer_Perceptron(input_dim = 3,intern_dim = 64, output_dim = 1, depth = 2, isBiased = True), env = gym.make('MountainCar-v0'), gamma = 0.99, replay_buffer_SIZE = 10000, batch_size = 64, observation_SIZE = 6, optimizer = torch.optim.AdamW):
        Agent.__init__(self,id,env)
        self.Q = Q
        self.gamma = gamma
        self.epsilon = epsilon
        self.replay_buffer = np.zeros((replay_buffer_SIZE,observation_SIZE))
        self.batch_size = batch_size
        self.optimizer = optimizer(self.Q.parameters())
        self.replay_buffer_SIZE = replay_buffer_SIZE
        self.observation_SIZE = observation_SIZE
        
    def observe(self, state, action, next_state, reward):
        state = np.array(state)
        action = np.array([action])
        next_state = np.array(next_state)
        reward = np.array([reward])

        concatenatation = np.concatenate((state, action, next_state, reward))
        return concatenatation
        
    def select_action(self, state):
        return random.randint(0,1)
        
        
    def update(self):

        LossFct = DQN_Loss()
        
        batch = np.random.choice(self.replay_buffer.shape[0], self.batch_size)

        for i in range(self.batch_size):

            A0 = torch.from_numpy(np.concatenate((self.replay_buffer[batch[i]][3:5], np.array([0])))).to(torch.float32)
            A1 = torch.from_numpy(np.concatenate((self.replay_buffer[batch[i]][3:5], np.array([1])))).to(torch.float32)
            target = self.replay_buffer[batch[i]][5] + self.gamma*max(self.Q(A0),self.Q(A1))
            input = self.Q(torch.from_numpy(self.replay_buffer[batch[i]][:3]).to(torch.float32))

            loss = LossFct(input, target)
            
            self.optimizer.zero_grad()
            loss.backward()
            #print(loss.item())
            self.optimizer.step()
        

    def train(self, episodes, debug_mode=False):
        episodesHistory = np.zeros((episodes))
        
        for i in range(episodes):
            if debug_mode: print("Episode: "+str(i+1)+" starts")
            newSeed = random.randint(0,100000)
            state,_ = self.env.reset(seed = newSeed)
            done = False
            episode_reward = 0
            self.replay_buffer = np.zeros((self.replay_buffer_SIZE, self.observation_SIZE))
            k=0
            
            while not done:
                                        
                action = self.select_action(state)
                next_state, reward, terminated, truncated, _ = self.env.step(action)
                observe = self.observe(state,action,next_state,reward)
                self.replay_buffer[k] = observe
                k+=1

                episode_reward += reward
                state = next_state
                done = terminated or truncated

            if debug_mode: print("Episode "+str(i+1)+ " , Reward: "+str(episode_reward))
            self.update()
            episodesHistory[i] = episode_reward
        if debug_mode: print(episodesHistory[:])
        return episodesHistory[:]

In [90]:
DQN = DQNAgent(0,0.9)
DQN.train(100, True)

Episode: 1 starts
Episode 1 , Reward: -200.0
Episode: 2 starts
Episode 2 , Reward: -200.0
Episode: 3 starts
Episode 3 , Reward: -200.0
Episode: 4 starts
Episode 4 , Reward: -200.0
Episode: 5 starts
Episode 5 , Reward: -200.0
Episode: 6 starts
Episode 6 , Reward: -200.0
Episode: 7 starts
Episode 7 , Reward: -200.0
Episode: 8 starts
Episode 8 , Reward: -200.0
Episode: 9 starts
Episode 9 , Reward: -200.0
Episode: 10 starts
Episode 10 , Reward: -200.0
Episode: 11 starts
Episode 11 , Reward: -200.0
Episode: 12 starts
Episode 12 , Reward: -200.0
Episode: 13 starts
Episode 13 , Reward: -200.0
Episode: 14 starts
Episode 14 , Reward: -200.0
Episode: 15 starts
Episode 15 , Reward: -200.0
Episode: 16 starts
Episode 16 , Reward: -200.0
Episode: 17 starts
Episode 17 , Reward: -200.0
Episode: 18 starts
Episode 18 , Reward: -200.0
Episode: 19 starts
Episode 19 , Reward: -200.0
Episode: 20 starts
Episode 20 , Reward: -200.0
Episode: 21 starts
Episode 21 , Reward: -200.0
Episode: 22 starts
Episode 22 ,

array([-200., -200., -200., -200., -200., -200., -200., -200., -200.,
       -200., -200., -200., -200., -200., -200., -200., -200., -200.,
       -200., -200., -200., -200., -200., -200., -200., -200., -200.,
       -200., -200., -200., -200., -200., -200., -200., -200., -200.,
       -200., -200., -200., -200., -200., -200., -200., -200., -200.,
       -200., -200., -200., -200., -200., -200., -200., -200., -200.,
       -200., -200., -200., -200., -200., -200., -200., -200., -200.,
       -200., -200., -200., -200., -200., -200., -200., -200., -200.,
       -200., -200., -200., -200., -200., -200., -200., -200., -200.,
       -200., -200., -200., -200., -200., -200., -200., -200., -200.,
       -200., -200., -200., -200., -200., -200., -200., -200., -200.,
       -200.])